In [1]:
from google.colab import drive

drive.mount("/content/drive")
!mkdir -p ~/.kaggle/
!cp ./drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

Mounted at /content/drive


In [2]:
!kaggle datasets download -d marquis03/marquis-viton-hd

Dataset URL: https://www.kaggle.com/datasets/marquis03/marquis-viton-hd
License(s): Apache 2.0
100% 4.18G/4.19G [00:59<00:00, 126MB/s] 
100% 4.19G/4.19G [00:59<00:00, 75.2MB/s]


In [4]:
!mkdir data
!unzip -q -d data marquis-viton-hd.zip

!rm marquis-viton-hd.zip
drive.flush_and_unmount()

# Importing and Utils

In [30]:
import os
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2


random.seed(42)
data_root = "/content/data"

train_folder = os.path.join(data_root, "train")
test_folder = os.path.join(data_root, "test")

In [31]:
train_file_names = os.listdir(os.path.join(train_folder, "image"))
random.shuffle(train_file_names)

# Dataset

In [32]:
IMG_SIZE = (384, 512)

train_transforms = A.Compose([
    A.Resize(height=IMG_SIZE[0], width=IMG_SIZE[1]),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [33]:
BATCH_SIZE = 32

class TrainDataset(Dataset):
  def __init__(self, image_list, image_folder, transforms):
    self.image_list = image_list
    self.image_folder = image_folder
    self.transforms = transforms

  def __len__(self): return len(self.image_list)

  def __getitem__(self, i):
    image_name = self.image_list[i]

    agnostic_path = os.path.join(self.image_folder, "agnostic", image_name)
    cloth_path = os.path.join(self.image_folder, "cloth", image_name)
    output_img_path = os.path.join(self.image_folder, "image", image_name)

    agnostic_image = cv2.cvtColor(cv2.imread(agnostic_path), cv2.COLOR_BGR2RGB)
    cloth_image = cv2.cvtColor(cv2.imread(cloth_path), cv2.COLOR_BGR2RGB)
    output_img_image = cv2.cvtColor(cv2.imread(output_img_path), cv2.COLOR_BGR2RGB)

    agnostic_image = self.transforms(image=agnostic_image)["image"]
    cloth_image = self.transforms(image=cloth_image)["image"]
    output_img_image = self.transforms(image=output_img_image)["image"]

    return agnostic_image, cloth_image, output_img_image

train_dataset = TrainDataset(train_file_names, train_folder, train_transforms)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [34]:
a, b, c = next(iter(train_dataloader))

# Model

In [50]:
class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
    super(DoubleConv, self).__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.padding = padding

    self.double_conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding),
        nn.ReLU(inplace=True)
    )

  def forward(self, X):
    X = self.double_conv(X)

    return X

In [78]:
class DownSample(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
    super(DownSample, self).__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.padding = padding

    self.double_conv = DoubleConv(in_channels, out_channels, kernel_size=kernel_size, padding=padding)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

  def forward(self, X):
    X = self.double_conv(X)
    X_pooled = self.pool(X)

    return X, X_pooled

In [77]:
class UpSample(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
    super(UpSample, self).__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.padding = padding

    self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
    self.double_conv = DoubleConv(in_channels, out_channels, kernel_size=kernel_size, padding=padding)

  def forward(self, X, X_skip):
    X = self.up(X)

    X_cat = torch.cat((X, X_skip), dim=1)
    X_cat = self.double_conv(X_cat)

    return X_cat

In [91]:
class UNet(nn.Module):
  def __init__(self, filter_sizes):
    super(UNet, self).__init__()

    self.filter_sizes = filter_sizes
    self.down_sample_blocks = []

    current_in_channels = 6
    for filter_size in filter_sizes:
      self.down_sample_blocks.append(DownSample(current_in_channels, filter_size, kernel_size=3, padding=1))
      current_in_channels = filter_size

    self.bottleneck = DoubleConv(filter_sizes[-1], filter_sizes[-1]*2, kernel_size=3, padding=1)

    self.up_sample_blocks = []
    for filter_size in filter_sizes[::-1]:
      self.up_sample_blocks.append(UpSample(filter_size*2, filter_size, kernel_size=3, padding=1))

    self.out_conv = nn.Conv2d(filter_sizes[0], 3, kernel_size=1)


  def forward(self, X_agnostic, X_cloth):
    X = torch.cat((X_agnostic, X_cloth), dim=1)

    X1_skip, X = self.down_sample_blocks[0](X)
    X2_skip, X = self.down_sample_blocks[1](X)
    X3_skip, X = self.down_sample_blocks[2](X)
    X4_skip, X = self.down_sample_blocks[3](X)

    X = self.bottleneck(X)

    X = self.up_sample_blocks[0](X, X4_skip)
    X = self.up_sample_blocks[1](X, X3_skip)
    X = self.up_sample_blocks[2](X, X2_skip)
    X = self.up_sample_blocks[3](X, X1_skip)

    X = self.out_conv(X)

    return X

In [93]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNet([16, 32, 64, 128]).to(device)

# Training